In [ ]:
import sys
import os
import csv
import pandas as pd

import mysql.connector
#import pymsql

import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
#geopy.geocoders.options.default_user_agent = "my_request"

In [ ]:
DIR_PATH = os.path.dirname(__file__)
FILE_PATH = os.path.join(DIR_PATH, "DataEngineer2022_corrected.sql")

path = os.getcwd()
csv_file = path + '\\DataEngineer2022_corrected.sql'

### MySQL connexion

In [152]:
con = mysql.connector.connect(host="localhost", user="root", password="", database="dataengineer",)
cursor = con.cursor()

query = ('ALTER TABLE address ADD COLUMN (latitude VARCHAR(100), longitude VARCHAR(100));')

try:
    cursor.execute(query)
    con.commit()

except :
    con.rollback()
    
cursor.close()
con.close()

In [ ]:
con = mysql.connector.connect(host="localhost", user="root", password="", database="dataengineer",)

cursor = con.cursor()
sql_query = "SELECT * FROM address"
df = pd.read_sql(sql_query, con)

df['location'] = df['address'].apply(lambda x: ' '.join([x.split(' ')[0], x.split(' ')[-1]])) #[-2:]
df['location'] = df['location'] + ', ' + df.city + ', ' + df.postal_code + ', FRANCE'

In [54]:
geolocator = Nominatim(user_agent="my_request") # Create an instance of Nominatim class
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1) # Wrap instance for query rate limitation

try:
    df['geocode'] = df['location'].apply(geocode) # Applying geocode method to get location into DataFrame

except Exception as inst:
    print(inst)

df['Lat'] = df['geocode'].apply(lambda x: x.latitude if x else '') #if x != None else None
df['Lon'] = df['geocode'].apply(lambda x: x.longitude if x else '')
df = df.fillna("")

RateLimiter caught an error, retrying (0/2 tries). Called with (*('29 RUE DE BOD AVEL, LOCMINE, 56500, France',), **{}).
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "C:\ProgramData\Anaconda3\lib\site-packages\urllib3\util\connection.py", line 95, in create_connection
    raise err
  File "C:\ProgramData\Anaconda3\lib\site-packages\urllib3\util\connection.py", line 85, in create_connection
    sock.connect(sa)
socket.timeout: timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py", line 703, in urlopen
    httplib_response = self._make_request(
  File "C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py", line 386, in _make_request
    self._validate_conn(conn)
  File "C:\ProgramData\Anaconda3\lib\sit

In [58]:
df.to_csv('address_geocode.csv', index=False, sep=';', encoding='utf-8')  

In [158]:
df.geocode.isna().sum(), df.shape

(101, (562, 9))

In [ ]:
# A NE PAS FAIRE  !!!!
# Plutôt executer un UPDATE dataengineer SET address ...
# Ne pas effacer car la table car relation avec d'autres tables

query = "DELETE FROM address WHERE address_id IS NOT NULL;"


try:
    con = mysql.connector.connect(host="localhost", user="root", password="", database="dataengineer")
    cursor = con.cursor()
    cursor.execute(query)
    con.commit()

finally:
    cursor.close()
    con.close()

In [ ]:
data = df[['address_id', 'address', 'city', 'postal_code', 'Lat', 'Lon']]
data = data.fillna("")

val_to_insert = data.values.tolist()

try:
    con = mysql.connector.connect(host="localhost", user="root", password="", database="dataengineer")
    cursor = con.cursor()
    
    cursor.executemany("INSERT INTO address(address_id, address, city, postal_code, latitude, longitude) \
    values (%s, %s, %s, %s, %s, %s);", val_to_insert)
    
    con.commit()
    
finally:
    cursor.close()
    con.close()

### Nominatim API

In [ ]:
SELECT * FROM suppliers WHERE state = 'Florida' AND supplier_id > 1000;

dd = ("SELECT suppliers.suppler_name, orders.order_id FROM suppliers, orders \
WHERE suppliers.supplier_id = orders.supplier_id AND suppliers.state = 'California';")

query4 = 'SELECT last_name, first_name, count(rental from customer_id), \ 
adress_id, latitude, longitue FROM actor, rental;'

In [ ]:
def read_dump(filename):
    with open(filename, 'r') as f:
        for line in f:
            cursor.execute(line)

https://blog.adrienvh.fr/2015/01/18/geocoder-en-masse-plusieurs-milliers-dadresses-avec-python-et-nominatim/

https://www.youtube.com/watch?v=0IjdfgmWzMk

https://www.askpython.com/python/python-geopy-to-find-geocode-of-an-address

https://github.com/tiffsea/python-code/blob/master/demos/geopy_query_addresses.ipynb

In [ ]:
C:\xampp\mysql\bin>mysql -u {DB_USER} -p {DB_NAME} < path/to/file/ab.sql
xampp shell: mysql -h localhost -u root databasename < dump.sql
    
https://simplebackups.com/blog/how-to-import-sql-file-in-mysql/
$ mysqlimport -u magazine_admin -p magazines_production ~/backup/database/magazines.sql